# NFPD data exploration

> These are Go notebooks: In order to use the GoNB Jupyter Kernel, please install GoNB from here: https://github.com/janpfeifer/gonb

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/anglersim=/path/to/anglersim"` at the top of any cell.

In this notebook we're exploring the data from the NFPD bulk download facility.

In [ ]:
!*go mod edit -replace "github.com/umbralcalc/anglersim=/home/robert/Code/anglersim"

In [ ]:
import (
	"os"
	"github.com/umbralcalc/anglersim/pkg/nfpd"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

sdf := nfpd.GetUniqueSitesDataFrameFromCountsCSV("../dat/FW_Fish_Counts.csv")
fmt.Println(sdf)

f, err := os.Create("../dat/FW_Fish_Unique_Count_Sites.csv")
if err != nil {
	panic(err)
}
sdf.WriteCSV(f)

In [ ]:
import (
	"gonum.org/v1/gonum/floats"
	"github.com/umbralcalc/anglersim/pkg/nfpd"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

siteName := "Haxted Mill"
fdf := nfpd.GetSiteCountsDataFrameFromCSV("../dat/FW_Fish_Counts.csv", siteName)

scatter := analysis.NewScatterPlotFromDataFrame(
	&fdf,
	"TIMESTAMP",
	"ALL_RUNS",
	"SPECIES_NAME",
)

xAxis := fdf.Col("TIMESTAMP").Float()
yAxis := fdf.Col("ALL_RUNS").Float()
scatter.SetGlobalOptions(
	charts.WithTitleOpts(opts.Title{
		Title: "Site: " + siteName,
		Bottom: "1%",
	}),
	charts.WithYAxisOpts(opts.YAxis{
		Min: floats.Min(yAxis),
		Max: floats.Max(yAxis),
	}),
	charts.WithXAxisOpts(opts.XAxis{
		Min: floats.Min(xAxis),
		Max: floats.Max(xAxis),
	}),
)

gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

In [ ]:
import (
    "github.com/umbralcalc/anglersim/pkg/nfpd"
)

%%

df := nfpd.GetTypesDataFrameFromCSV("../dat/FW_Fish_Data_Types.csv")

fmt.Println(df)

In [ ]:
import (
    "github.com/umbralcalc/anglersim/pkg/nfpd"
)

%%

df := nfpd.GetSitesDataFrameFromCSV("../dat/FW_Fish_Sites.csv")

fmt.Println(df)

In [ ]:
import (
    "bufio"
	"bytes"
	"fmt"
	"os"

	"github.com/go-gota/gota/dataframe"
)

%%

const maxRows = 100

f, err := os.Open("../dat/FW_Fish_Individual_Lengths.csv")
if err != nil {
	panic(err)
}

scanner := bufio.NewScanner(f)
var buffer bytes.Buffer

if scanner.Scan() {
	buffer.WriteString(scanner.Text() + "\n")
} else {
	panic("Empty file")
}

count := 0
for scanner.Scan() {
	buffer.WriteString(scanner.Text() + "\n")
	count++
	if count >= maxRows {
		break
	}
}
if err := scanner.Err(); err != nil {
	panic(err)
}

df := dataframe.ReadCSV(bytes.NewReader(buffer.Bytes()))

fmt.Println(df)

In [ ]:
import (
    "os"
	"github.com/go-gota/gota/dataframe"
)

%%

f, err := os.Open("../dat/FW_Fish_Banded_Measurements.csv")
if err != nil {
	panic(err)
}

df := dataframe.ReadCSV(f)

fmt.Println(df)

In [ ]:
import (
    "os"
	"github.com/go-gota/gota/dataframe"
)

%%

f, err := os.Open("../dat/FW_Fish_Bulk_Measurements.csv")
if err != nil {
	panic(err)
}

df := dataframe.ReadCSV(f)

fmt.Println(df)